In [0]:
import math
import numpy as np
import tensorflow as tf

import pandas as pd

In [2]:

def generate_cylinder_dataset(size):
  base_radius = np.random.uniform(low=0.5, high=2.0, size=size)
  height = np.random.uniform(low=0.5, high=2.0, size=size)
  volume = math.pi * (base_radius ** 2) * height
  return pd.DataFrame({'base_radius': base_radius, 'height': height, 'volume': volume})

cylinder_train_df = generate_cylinder_dataset(500000)
cylinder_test_df = generate_cylinder_dataset(1000)
print cylinder_train_df.head()

   base_radius    height     volume
0     1.940294  1.423212  16.832725
1     0.624015  1.605018   1.963448
2     1.358647  1.782694  10.338073
3     1.627153  1.730534  14.394174
4     1.938898  1.339364  15.818245


In [0]:
feature_columns = [
  tf.feature_column.numeric_column("base_radius"),
  tf.feature_column.numeric_column("height")
]

def make_input_fn(df, num_epochs, batch_size=512):
  return tf.estimator.inputs.pandas_input_fn(
    df,
    df['volume'],
    batch_size=batch_size,
    num_epochs=num_epochs,
    shuffle=True
  )

def predict_input_fn():
  features = {
    "base_radius": tf.constant([1.5]),
    "height": tf.constant([2.0])
  }
  return features

In [5]:
model = tf.estimator.DNNRegressor(
  feature_columns=feature_columns,
  hidden_units=[64, 32, 32, 16],
  optimizer=tf.train.AdagradOptimizer(learning_rate=0.05)
)
model.train(make_input_fn(cylinder_train_df, None, 1000), max_steps=5000)
predictions = model.predict(predict_input_fn)
print predictions.next()
print math.pi * (1.5 ** 2) * 2.0

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc00244a1d0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmps7_H8Z', '_train_distribute': None, '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmps7_H8Z/model.ckpt.
INFO:tensorf

In [6]:
def print_rmse(model, name, df):
  metrics = model.evaluate(input_fn = make_input_fn(cylinder_test_df, 1))
  print('RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss'])))
print_rmse(model, 'test', cylinder_test_df)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-29-16:31:21
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmps7_H8Z/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-29-16:31:21
INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.044495843, global_step = 1000, label/mean = 6.8507185, loss = 22.247921, prediction/mean = 6.6987953
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmps7_H8Z/model.ckpt-1000
RMSE on test dataset = 0.210940375924


Copyright 2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License